# Tutorial 7. Structuring Codebases

:::{note}
:icon: false

#### Imperative vs declarative, and why we recommend class-based APIs

As your app grows, two things become hard quickly: managing state and keeping behavior understandable.
In this tutorial, we compare imperative and declarative approaches, then build a composable class-based app architecture that scales.

## Learning goals

By the end, you should be able to:

1. Explain the difference between imperative and declarative reactive code.
2. Choose the right tool (`.watch`, `pn.bind`, `param.rx`) for a given task.
3. Structure a larger app using composable classes with clear responsibilities.

:::

---

In [ ]:
import numpy as np
import pandas as pd
import panel as pn
import panel_material_ui as pmui
import param

pn.extension("tabulator", throttled=True)


## 1) Imperative and declarative: what each buys you

### Imperative

Imperative code answers: **"When X changes, run Y."**

- Great for side effects (logging, saving files, notifications)
- Good for one-off interactions
- Can become hard to reason about as callback count grows

### Declarative

Declarative code answers: **"This output is a function of these inputs."**

- Great for UI/data transformations
- Easier to test and compose
- Reduces hidden state and callback chains

### Recommendation

For larger apps, default to **declarative + class-based composition**.  
Use imperative watchers for true side effects only.

---


## 2) A tiny imperative baseline


In [ ]:
import panel as pn
import panel_material_ui as pmui

pn.extension('tabulator')

slider = pmui.IntSlider(name="Threshold", start=0, end=100, value=50)
status = pn.pane.Markdown()

def _update_status(event):
    status.object = f"Current threshold: **{event.new}**"

slider.param.watch(_update_status, "value")
_update_status(type("E", (), {"new": slider.value})())  # initialize once

pn.Column(slider, status)

This is valid and useful. But if the app adds many widgets and many watchers, you can end up with scattered logic.

### Mini Exercise 1 (2 minutes)

Add a second slider (`Max threshold`) and keep the status message valid:

- show `"Invalid range"` when `min > max`
- otherwise show `"Range: min - max"`

Implement this first imperatively with `.param.watch`:

## 3) The same idea, declaratively

Now we encode output as a function of inputs:


In [ ]:
min_slider = pmui.IntSlider(name="Min", start=0, end=100, value=20)
max_slider = pmui.IntSlider(name="Max", start=0, end=100, value=80)

def describe_range(min_value, max_value):
    if min_value > max_value:
        return "### Invalid range"
    return f"### Range: {min_value} - {max_value}"

status = pn.bind(describe_range, min_slider, max_slider)

pn.Column(min_slider, max_slider, pn.pane.Markdown(status))

`pn.bind` makes dependencies explicit and keeps behavior in one place.

### Mini Exercise 2 (3 minutes)

Extend `describe_range` so it also returns:

- the span (`max - min`)
- a label `"narrow"` if span < 20 else `"wide"`


## 4) Why class-based composition scales better

When apps grow, classes give you:

- **Clear ownership**: one class per concern (filters, data, view, app shell)
- **Reusable units**: views can share one datastore
- **Testable logic**: state and transforms are isolated
- **Declarative dependencies**: methods depend on parameters, not global state

Below is a recommended pattern using small composable classes.

### Step 1: sample data

In [ ]:
import numpy as np
import pandas as pd

def make_data(n=1500, seed=42):
    rng = np.random.default_rng(seed)
    manufacturers = np.array(["Nordex", "Vestas", "Siemens", "Enercon"])
    years = rng.integers(2000, 2025, n)
    capacity = rng.normal(2800, 900, n).clip(200, 7000)
    manufacturer = rng.choice(manufacturers, n, replace=True)
    return pd.DataFrame(
        {"year": years, "capacity_kw": capacity.round(0), "manufacturer": manufacturer}
    )

### Step 2: a `Filters` class (state only)

In [ ]:
import param

class Filters(pn.viewable.Viewer):
    year = param.Range(default=(2010, 2020), bounds=(2000, 2025))
    manufacturers = param.ListSelector(default=[], objects=[])

    @classmethod
    def from_data(cls, df):
        f = cls()
        year_bounds = (int(df["year"].min()), int(df["year"].max()))
        f.param.year.bounds = year_bounds
        f.year = year_bounds
        objects = sorted(df["manufacturer"].unique().tolist())
        f.param.manufacturers.objects = objects
        f.manufacturers = objects
        return f

    def __panel__(self):
        return pn.Param(
            self,
            parameters=["year", "manufacturers"],
            widgets={"manufacturers": {"type": pmui.MultiChoice}},
            width=320,
        )


### Step 3: a `DataStore` class (data transforms)

In [ ]:
class DataStore(param.Parameterized):
    data = param.DataFrame()
    filters = param.ClassSelector(class_=Filters)

    @param.depends("data", "filters.year", "filters.manufacturers")
    def filtered(self):
        low, high = self.filters.year
        df = self.data
        mask = df["year"].between(low, high)
        if self.filters.manufacturers:
            mask &= df["manufacturer"].isin(self.filters.manufacturers)
        return df.loc[mask]

### Step 4: view classes (presentation only)

In [ ]:
class Indicators(pn.viewable.Viewer):
    data_store = param.ClassSelector(class_=DataStore)

    @param.depends("data_store.data", "data_store.filters.year", "data_store.filters.manufacturers")
    def __panel__(self):
        df = self.data_store.filtered()
        return pn.FlexBox(
            pn.indicators.Number(name="Rows", value=len(df), format="{value:,.0f}"),
            pn.indicators.Number(name="Avg capacity (kW)", value=df["capacity_kw"].mean(), format="{value:,.0f}"),
            gap="10px",
        )


class Table(pn.viewable.Viewer):
    data_store = param.ClassSelector(class_=DataStore)

    @param.depends("data_store.data", "data_store.filters.year", "data_store.filters.manufacturers")
    def __panel__(self):
        return pn.widgets.Tabulator(
            self.data_store.filtered(),
            pagination="remote",
            page_size=12,
            sizing_mode="stretch_width",
            height=360,
        )

### Step 5: app shell class (composition)

In [ ]:
class App(pn.viewable.Viewer):
    title = param.String(default="Wind Explorer")
    data_store = param.ClassSelector(class_=DataStore)

    def __panel__(self):
        main = pn.Column(
            Indicators(data_store=self.data_store),
            Table(data_store=self.data_store),
        )
        if pn.state.served:
            page = pmui.Page(title=self.title)
            page.sidebar.append(self.data_store.filters)
            page.main.append(main)
        else:
            page = pn.Row(
                self.data_store.filters,
                main
            )
        return page


### Run it

In [ ]:
df = make_data()
filters = Filters.from_data(df)
store = DataStore(data=df, filters=filters)
App(data_store=store).servable()

## 5) Where imperative still belongs

Keep imperative `.watch` for side effects, for example:

- writing logs
- persisting user settings
- triggering notifications

Example:

```python
def log_filter_change(event):
    print(f"[filters] {event.name} changed to {event.new}")

filters.param.watch(log_filter_change, ["year", "manufacturers"])
```

This separation works well:

- **Declarative** for data/UI derivations
- **Imperative** for side effects

## 7) Mini Exercises

### Mini Exercise 3: add a new view

Create a `ManufacturerBreakdown` view class that:

- groups filtered data by `manufacturer`
- shows sum of `capacity_kw`
- renders as a `Tabulator`

### Mini Exercise 4: test class boundaries

Move all filter widget configuration into `Filters.__panel__()` and keep `DataStore` free of widget code.

Ask yourself: can you unit test `DataStore.filtered()` without rendering any UI?

### Section Recap Exercise

Add an imperative watcher that logs changes to filter state, but keep all displayed outputs declarative.

## Recap

You now have a scalable structure for larger Panel applications:

- keep data/state in focused classes with clear responsibilities
- prefer declarative outputs for maintainable UI behavior
- use imperative hooks only where side effects and lifecycle concerns require them